In [ ]:
import pandas as pd
import numpy as np

# --- Load Dataset ---
df = pd.read_csv(
    r"D:\Project_Data\us_congestion_2016_2022_sample_2m\us_congestion_2016_2022_sample_2m.csv",
    encoding="utf-8",
    low_memory=False
)

# --- Cleaning Pipeline ---
"""
Cleans congestion dataset:
- Fixes datetime columns
- Handles missing values in weather-related columns
- Drops rows with missing critical traffic/location fields
- Creates derived features (Duration_min, numeric congestion speed)
"""

# --- 1. Fix datetime columns ---
for col in ['StartTime', 'EndTime', 'WeatherTimeStamp']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)

# --- 2. Derived feature: Duration (in minutes) ---
if 'StartTime' in df.columns and 'EndTime' in df.columns:
    df['Duration_min'] = (df['EndTime'] - df['StartTime']).dt.total_seconds() / 60
    df['Duration_min'] = df['Duration_min'].fillna(0)

# --- 3. Numeric conversion ---
num_cols = [
    'Severity', 'Distance(mi)', 'DelayFromTypicalTraffic(mins)',
    'DelayFromFreeFlowSpeed(mins)', 'Temperature(F)', 'WindChill(F)',
    'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'WindSpeed(mph)',
    'Precipitation(in)'
]
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# --- 4. Handle Missing Values ---
# Drop rows if critical columns are missing
critical_cols = [
    'ID', 'City', 'State', 'Country', 'StartTime', 'EndTime',
    'DelayFromTypicalTraffic(mins)', 'Congestion_Speed'
]
df = df.dropna(subset=[col for col in critical_cols if col in df.columns])

# Weather-related columns
if 'Temperature(F)' in df.columns:
    df['Temperature(F)'] = df.groupby('City')['Temperature(F)'].transform(lambda x: x.fillna(x.mean()))
if 'Humidity(%)' in df.columns:
    df['Humidity(%)'] = df.groupby('City')['Humidity(%)'].transform(lambda x: x.fillna(x.mean()))
if 'Weather_Event' in df.columns:
    df['Weather_Event'] = df['Weather_Event'].fillna("None")
if 'Weather_Conditions' in df.columns:
    df['Weather_Conditions'] = (
        df['Weather_Conditions'].astype(str).str.strip().str.lower().replace("nan", np.nan).fillna("Unknown")
    )

# --- 5. Numeric encoding for Congestion_Speed ---
speed_map = {'Very Slow': 5, 'Slow': 10, 'Moderate': 20, 'Fast': 30}
if 'Congestion_Speed' in df.columns:
    df['Congestion_Speed_num'] = df['Congestion_Speed'].map(speed_map)

# --- 6. Drop duplicates by ID ---
df = df.drop_duplicates(subset=['ID'])

# --- 7. Save Cleaned Dataset ---
output_path = r"D:\Project_Data\Processed Data\cleaned_congestion.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print(f"Cleaning complete. Saved cleaned dataset to: {output_path}")